In [9]:
import re
import requests
from bs4 import BeautifulSoup
import time

import selenium
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")
%matplotlib inline

In [10]:
# district = 'Академический,Алексеевский,Алтуфьевский,Арбат,Аэропорт,Бабушкинский,Басманный,Беговой,Бескудниковский,Бибирево,Бирюлёво Восточное,Бирюлёво Западное,Богородское,Братеево,Бутово Северное,Бутово Южное,Бутырский.Вешняки,Внуково,Войковский,Восточный,Выхино-Жулебино.Гагаринский,Головинский,Гольяново,Даниловский,Дегунино Восточное,Дегунино Западное,Дмитровский,Донской,Дорогомилово.Замоскворечье,Зюзино,Зябликово.Ивановское,Измайлово Восточное,Измайлово,Измайлово Северное,Капотня,Коньково,Коптево,Косино-Ухтомский,Котловка,Красносельский,Крылатское,Крюково,Кузьминки,Кунцево,Куркино.Левобережный,Лефортово,Лианозово,Ломоносовский,Лосиноостровский,Люблино.Марфино,Марьина роща,Марьино,Матушкино,Медведково Северное,Медведково Южное,Метрогородок,Мещанский,Митино,Можайский,Молжаниновский,Москворечье-Сабурово.Нагатино-Садовники,Нагатинский затон,Нагорный,Некрасовка,Нижегородский,Ново-Переделкино,Новогиреево,Новокосино.Обручевский,Орехово-Борисово Северное,Орехово-Борисово Южное,Останкинский,Отрадное,Очаково-Матвеевское.Перово,Печатники,Покровское-Стрешнево,Преображенское,Пресненский,Проспект Вернадского.Раменки,Ростокино,Рязанский.Савёлки,Савёловский,Свиблово,Северный,Силино,Сокол,Соколиная гора,Сокольники,Солнцево,Старое Крюково,Строгино.Таганский,Тверской,Текстильщики,Тёплый Стан,Тимирязевский,Тропарёво-Никулино,Тушино Северное,Тушино Южное.Филёвский парк,Фили-Давыдково.Хамовники,Ховрино,Хорошёво-Мневники,Хорошёвский.Царицыно,Черёмушки,Чертаново Северное,Чертаново Центральное,Чертаново Южное.Щукино.Южнопортовый.Якиманка,Ярославский,Ясенево'
# districts = re.split(',|\\.',district)

districts = 'Поселение Киевское, Филевский парк, Поселение Новофедоровское, Поселение Роговское, Поселение Мосрентген, Поселение Вороновское, Поселение Михайлово-Ярцевское, Поселение Марушкинское, Поселение Первомайское, Матушкино, Внуково, Савельки, Поселение Внуковское, Силино, Поселение Кокошкино, «Крюково», «Щукино», «Поселение Краснопахорское», «Нагатинский Затон», «Старое Крюково», «Поселение Кленовское», Дмитровское, Поселение Филимонковское, Троицкий округ, «Теплый стан», «Поселение Щаповское», «Поселение Московский», Поселение Десёновское, Ховрино, Ломоносовское, Можайское, «Ново-Переделкино», «Строгино», «Молжаниновское», «Митино», Куркино, Крылатское, Солнцево, Поселение Сосенское, Поселение Воскресенское, Головинское, Южное Тушино, «Северное Тушино», «Черемушки», «Покровское Стрешнево», «Хорошево-Мневники», «Очаково-Матвеевское», «Тропарево-Никулино», «Левобережное», «Фили Давыдково», «Обручевское», Поселение Рязановское, Раменки, Войковское, Сокол, «Западное Дегунино», «Проспект Вернадского», «Южное Бутово», «Ясенево», «Дорогомилово», «Коньково», «Хорошевское», «Беговое», «Коптево», «Поселение Щербинка», «Аэропорт», «Пресненское», «Северное», «Бескудниковское», «Гагаринское», «Тимирязевское», «Северное Бутово», «Лианозово», «Хамовники», «Восточное Дегунино», «Савеловское», «Академическое», «Зюзино», «Алтуфьевское», «Марфино», «Чертаново Центральное», «Отрадное», «Арбат», «Чертаново Южное», «Бутырское», «Тверское», «Чертаново Северное», «Якиманка», «Котловка», «Останкинское», Донское, Бибирево, Бирюлево Западное, Марьина Роща, «Нагорное», «Свиблово», «Даниловское», «Мещанское», «Южное Медведково», «Замоскворечье», «Северное Медведково», «Нагатино-Садовники», «Москворечье-Сабурово», «Царицыно», «Басманное», «Красносельское», «Ростокино», «Таганское», «Алексеевское», «Сокольники», «Бирюлево Восточное», «Бабушкинское», «Южнопортовое», «Ярославское», «Печатники», Богородское, Метрогородок, Лефортово, «Орехово-Борисово Северное», «Лосиноостровское», «Нижегородское», «Перово», «Орехово-Борисово Южное», «Марьино», «Вешняки», «Преображенское», «Соколиная гора», «Люблино», «Текстильщики», Братеево, Зябликово, Рязанский, Измайлово, Новогиреево, «Кузьминки», «Гольяново», «Северное Измайлово», «Капотня», Выхино-Жулебино, Восточное, Восточное Измайлово, «Ивановское», «Косино-Ухтомское», «Новокосино», «Некрасовка», «Кунцево»'\
.replace(', ',  ",").replace('«', "").replace('»', "").split(",")

for i in range(len(districts)):
    if "Поселение" not in districts[i]:
        districts[i] = districts[i].replace('ое', "ий")
print(*districts, sep="\n")

Поселение Киевское
Филевский парк
Поселение Новофедоровское
Поселение Роговское
Поселение Мосрентген
Поселение Вороновское
Поселение Михайлово-Ярцевское
Поселение Марушкинское
Поселение Первомайское
Матушкино
Внуково
Савельки
Поселение Внуковское
Силино
Поселение Кокошкино
Крюково
Щукино
Поселение Краснопахорское
Нагатинский Затон
Старий Крюково
Поселение Кленовское
Дмитровский
Поселение Филимонковское
Троицкий округ
Теплый стан
Поселение Щаповское
Поселение Московский
Поселение Десёновское
Ховрино
Ломоносовский
Можайский
Ново-Переделкино
Строгино
Молжаниновский
Митино
Куркино
Крылатский
Солнцево
Поселение Сосенское
Поселение Воскресенское
Головинский
Южний Тушино
Северний Тушино
Черемушки
Покровский Стрешнево
Хорошево-Мневники
Очаково-Матвеевский
Тропарево-Никулино
Левобережний
Фили Давыдково
Обручевский
Поселение Рязановское
Раменки
Войковский
Сокол
Западний Дегунино
Проспект Вернадского
Южний Бутово
Ясенево
Дорогомилово
Коньково
Хорошевский
Беговий
Коптево
Поселение Щербинка
Аэропор

In [11]:
for i, n in enumerate([77]):
    print(i, n, districts[n])

0 77 Зюзино


In [14]:
driver = webdriver.Chrome('/Users/fuckingbell/chromedriver')
cian_Moscow = 'https://www.cian.ru'
# for district in districts[105:]:
for district in districts:
    driver.get(cian_Moscow)

    #openning  a combobox
    rooms_button = '/html/body/main/section/div/div[1]/div[2]/div[1]/div[1]/div[1]/div/div[2]/div/div[2]/button'
    wrapper = driver.find_element_by_xpath(rooms_button)
    wrapper.click()

    #rooms 3 4 5 6
    rooms = driver.find_elements_by_class_name('_025a50318d--button--39rK7')
    for room in rooms[2:]:
        room.click()
    # Studio apartment & free layout
    extended_rooms = driver.find_elements_by_class_name('_025a50318d--box--VTgYk')
    for room in extended_rooms:
        room.click()
        
    # rooms_3_4_5_6 = [
    #     '/html/body/main/section/div/div[1]/div[2]/div[1]/div[1]/div[1]/div/div[2]/div/div[2]/div/ul[1]/li[3]/button',
    #     '/html/body/main/section/div/div[1]/div[2]/div[1]/div[1]/div[1]/div/div[2]/div/div[2]/div/ul[1]/li[4]/button',
    #     '/html/body/main/section/div/div[1]/div[2]/div[1]/div[1]/div[1]/div/div[2]/div/div[2]/div/ul[1]/li[5]/button',
    #     '/html/body/main/section/div/div[1]/div[2]/div[1]/div[1]/div[1]/div/div[2]/div/div[2]/div/ul[1]/li[6]/button']
    # for room in rooms_3_4_5_6:
    #     select_room = driver.find_element_by_xpath(room)
    #     select_room.click()

    search  = driver.find_element_by_id("geo-suggest-input")
    if "Поселение" in district:
        search.send_keys(district + ' Москва')
    else:
        search.send_keys('район ' + district)
    # search.send_keys(Keys.RETURN)
    time.sleep(2)
    search.send_keys(Keys.RETURN)
    time.sleep(2)
    search.send_keys(Keys.RETURN)

    link = driver.find_element_by_link_text('Найти')
    link.click()
    time.sleep(4)

#     try:
#         time.sleep(2)    
#         driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

#     #     element = WebDriverWait(driver, 12).until(
#     #         EC.presence_of_element_located((By.CLASS_NAME, "_93444fe79c--main--1-d6V _93444fe79c--light--366j7"))
#     #     )

#         time.sleep(1)
#     #     element_2 = driver.find_element_by_partial_link_text('Excel')
#         element_2 = driver.find_element_by_class_name("_93444fe79c--main--1-d6V")
#         element_2.click()
#     except:
#         driver.quit()

#     time.sleep(6)

WebDriverException: Message: chrome not reachable
  (Session info: chrome=92.0.4515.131)


In [ ]:
driver = webdriver.Chrome('/Users/fuckingbell/chromedriver')
cian_Moscow = 'https://www.cian.ru'
for district in districts[100:]:
    driver.get(cian_Moscow)

    #openning  a combobox
    rooms_button = '/html/body/main/section/div/div[1]/div[2]/div[1]/div[1]/div[1]/div/div[2]/div/div[2]/button'
    wrapper = driver.find_element_by_xpath(rooms_button)
    wrapper.click()

    #rooms 3 4 5 6
    rooms = driver.find_elements_by_class_name('_025a50318d--button--39rK7')
    for room in rooms[2:]:
        room.click()
    # Studio apartment & free layout
    extended_rooms = driver.find_elements_by_class_name('_025a50318d--box--VTgYk')
    for room in extended_rooms:
        room.click()

    search  = driver.find_element_by_id("geo-suggest-input")
    search.send_keys(district + ' Москва')
    
    time.sleep(2)
    # search.send_keys(Keys.RETURN)
    search.send_keys(Keys.RETURN)
    time.sleep(2)
    search.send_keys(Keys.RETURN)

    link = driver.find_element_by_link_text('Найти')
    link.click()

    time.sleep(2)

    try:
        time.sleep(2)    
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        element_2 = driver.find_element_by_class_name("_93444fe79c--main--1-d6V")
        element_2.click()
    except:
        driver.quit()
    time.sleep(6)

In [25]:
needed_columns = ['ID', 'Количество комнат', 'Тип', 'Метро', 'Адрес', 'Площадь, м2', 'Дом', 'Цена', 'Телефоны',
                  'Ремонт', 'Площадь комнат, м2', 'Балкон', 'Окна', 'Санузел', 'Высота потолков, м']

In [12]:
file_it = pd.read_excel("/Users/fuckingbell/Programming/Pet Project/offers/offers (0).xlsx")

In [35]:
offers_data = pd.DataFrame()
for i in range(125):
    file_path = f"/Users/fuckingbell/Programming/Pet Project/offers/offers ({i}).xlsx"
    file_it = pd.read_excel(file_path)
    file_it['Район'] = 'Район ' + districts[i]
    offers_data = offers.append(file_it)

In [52]:
offers_data["price"] = offers_data["Цена"].str.replace(' .+', '').astype(float)
offers_data["square"] = offers_data["Площадь, м2"].str.replace('/.+', '').astype(float)

In [56]:
prices_data = offers_data.groupby("Район").agg({"price":"sum", "square":"sum"})
prices_data["average_price"] = prices_data.price / prices_data.square
prices_data

In [ ]:
,

In [ ]:
[104, 105, 107, 108, 109, 111, 113, 119, 120, 121, 122, 125, 140, 141, 142]

<>:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
<>:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
<ipython-input-33-aad641418de9>:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  [1, 2, 3, 4, 5][2, 4, 5, 6]


TypeError: list indices must be integers or slices, not tuple

In [1]:
"Центральный административный округ:,\
Районы ЦАО на карте Москвы,\
Арбат,\
Басманный район,\
Замоскворечье,\
Красносельский район,\
Мещанский район,\
Пресненский район,\
Таганский район,\
Тверской район,\
Хамовники,\
Якиманка,\
Северный административный округ:,\
Районы САО на карте Москвы,\
Аэропорт,\
Беговой,\
Бескудниковский район,\
Войковский район,\
Восточное Дегунино,\
Головинский район,\
Дмитровский район,\
Западное Дегунино,\
Коптево,\
Левобережный,\
Молжаниновский район,\
Савёловский район,\
Сокол,\
Тимирязевский район,\
Ховрино,\
Хорошёвский район,\
Северо-Восточный административный округ:,\
Районы СВАО на карте Москвы,\
Алексеевский район,\
Алтуфьевский район,\
Бабушкинский район,\
Бибирево,\
Бутырский район,\
Лианозово,\
Лосиноостровский район,\
Марфино,\
Марьина роща,\
Останкинский район,\
Отрадное,\
Ростокино,\
Свиблово,\
Северный,\
Северное Медведково,\
Южное Медведково,\
Ярославский район,\
Восточный административный округ:,\
Районы ВАО на карте Москвы,\
Богородское,\
Вешняки,\
Восточный,\
Восточное Измайлово,\
Гольяново,\
Ивановское,\
Измайлово,\
Косино-Ухтомский,\
Метрогородок,\
Новогиреево,\
Новокосино,\
Перово,\
Преображенское,\
Северное Измайлово,\
Соколиная Гора,\
Сокольники,\
Юго-Восточный административный округ:,\
Районы ЮВАО на карте Москвы,\
Выхино-Жулебино,\
Капотня,\
Кузьминки,\
Лефортово,\
Люблино,\
Марьино,\
Некрасовка,\
Нижегородский район,\
Печатники,\
Рязанский район,\
Текстильщики,\
Южнопортовый район,\
Южный административный округ:,\
Районы ЮАО на карте Москвы,\
Бирюлёво Восточное,\
Бирюлёво Западное,\
Братеево,\
Даниловский район,\
Донской район,\
Зябликово,\
Москворечье-Сабурово,\
Нагатино-Садовники,\
Нагатинский Затон,\
Нагорный район,\
Орехово-Борисово Северное,\
Орехово-Борисово Южное,\
Царицыно,\
Чертаново Северное,\
Чертаново Центральное,\
Чертаново Южное,\
Юго-Западный административный округ:,\
Районы ЮЗАО на карте Москвы,\
Академический район,\
Гагаринский район,\
Зюзино,\
Коньково,\
Котловка,\
Ломоносовский район,\
Обручевский район,\
Северное Бутово,\
Тёплый Стан,\
Черёмушки,\
Южное Бутово,\
Ясенево,\
Западный административный округ:,\
Районы ЗАО на карте Москвы,\
Внуково,\
Дорогомилово,\
Крылатское,\
Кунцево,\
Можайский район,\
Ново-Переделкино,\
Очаково-Матвеевское,\
Проспект Вернадского,\
Раменки,\
Солнцево,\
Тропарёво-Никулино,\
Филёвский Парк,\
Фили-Давыдково,\
Северо-Западный административный округ:,\
Районы СЗАО на карте Москвы,\
Куркино,\
Митино,\
Покровское-Стрешнево,\
Северное Тушино,\
Строгино,\
Хорошёво-Мнёвники,\
Щукино,\
Южное Тушино,\
Зеленоградский административный округ:,\
Районы ЗелАО на карте Москвы,\
Матушкино,\
Савёлки,\
Старое Крюково,\
Силино,\
Крюково".split(",")

['Центральный административный округ:',
 'Районы ЦАО на карте Москвы',
 'Арбат',
 'Басманный район',
 'Замоскворечье',
 'Красносельский район',
 'Мещанский район',
 'Пресненский район',
 'Таганский район',
 'Тверской район',
 'Хамовники',
 'Якиманка',
 'Северный административный округ:',
 'Районы САО на карте Москвы',
 'Аэропорт',
 'Беговой',
 'Бескудниковский район',
 'Войковский район',
 'Восточное Дегунино',
 'Головинский район',
 'Дмитровский район',
 'Западное Дегунино',
 'Коптево',
 'Левобережный',
 'Молжаниновский район',
 'Савёловский район',
 'Сокол',
 'Тимирязевский район',
 'Ховрино',
 'Хорошёвский район',
 'Северо-Восточный административный округ:',
 'Районы СВАО на карте Москвы',
 'Алексеевский район',
 'Алтуфьевский район',
 'Бабушкинский район',
 'Бибирево',
 'Бутырский район',
 'Лианозово',
 'Лосиноостровский район',
 'Марфино',
 'Марьина роща',
 'Останкинский район',
 'Отрадное',
 'Ростокино',
 'Свиблово',
 'Северный',
 'Северное Медведково',
 'Южное Медведково',
 'Яро

In [4]:
moscow_dic = {'Центральный административный округ':
 
 ['Арбат',
 'Басманный район',
 'Замоскворечье',
 'Красносельский район',
 'Мещанский район',
 'Пресненский район',
 'Таганский район',
 'Тверской район',
 'Хамовники',
 'Якиманка'],
 'Северный административный округ':

 ['Аэропорт',
 'Беговой',
 'Бескудниковский район',
 'Войковский район',
 'Восточное Дегунино',
 'Головинский район',
 'Дмитровский район',
 'Западное Дегунино',
 'Коптево',
 'Левобережный',
 'Молжаниновский район',
 'Савёловский район',
 'Сокол',
 'Тимирязевский район',
 'Ховрино',
 'Хорошёвский район'],
 'Северо-Восточный административный округ':

 ['Алексеевский район',
 'Алтуфьевский район',
 'Бабушкинский район',
 'Бибирево',
 'Бутырский район',
 'Лианозово',
 'Лосиноостровский район',
 'Марфино',
 'Марьина роща',
 'Останкинский район',
 'Отрадное',
 'Ростокино',
 'Свиблово',
 'Северный',
 'Северное Медведково',
 'Южное Медведково',
 'Ярославский район'],
 'Восточный административный округ':

 ['Богородское',
 'Вешняки',
 'Восточный',
 'Восточное Измайлово',
 'Гольяново',
 'Ивановское',
 'Измайлово',
 'Косино-Ухтомский',
 'Метрогородок',
 'Новогиреево',
 'Новокосино',
 'Перово',
 'Преображенское',
 'Северное Измайлово',
 'Соколиная Гора',
 'Сокольники'],
 'Юго-Восточный административный округ':

 ['Выхино-Жулебино',
 'Капотня',
 'Кузьминки',
 'Лефортово',
 'Люблино',
 'Марьино',
 'Некрасовка',
 'Нижегородский район',
 'Печатники',
 'Рязанский район',
 'Текстильщики',
 'Южнопортовый район'],
 'Южный административный округ':
 ['Бирюлёво Восточное',
 'Бирюлёво Западное',
 'Братеево',
 'Даниловский район',
 'Донской район',
 'Зябликово',
 'Москворечье-Сабурово',
 'Нагатино-Садовники',
 'Нагатинский Затон',
 'Нагорный район',
 'Орехово-Борисово Северное',
 'Орехово-Борисово Южное',
 'Царицыно',
 'Чертаново Северное',
 'Чертаново Центральное',
 'Чертаново Южное'],
 'Юго-Западный административный округ':
 ['Академический район',
 'Гагаринский район',
 'Зюзино',
 'Коньково',
 'Котловка',
 'Ломоносовский район',
 'Обручевский район',
 'Северное Бутово',
 'Тёплый Стан',
 'Черёмушки',
 'Южное Бутово',
 'Ясенево'],
 'Западный административный округ':
 ['Внуково',
 'Дорогомилово',
 'Крылатское',
 'Кунцево',
 'Можайский район',
 'Ново-Переделкино',
 'Очаково-Матвеевское',
 'Проспект Вернадского',
 'Раменки',
 'Солнцево',
 'Тропарёво-Никулино',
 'Филёвский Парк',
 'Фили-Давыдково'],
 'Северо-Западный административный округ':

 ['Куркино',
 'Митино',
 'Покровское-Стрешнево',
 'Северное Тушино',
 'Строгино',
 'Хорошёво-Мнёвники',
 'Щукино',
 'Южное Тушино'],
 'Зеленоградский административный округ':

 ['Матушкино',
 'Савёлки',
 'Старое Крюково',
 'Силино',
 'Крюково']}

In [6]:
sum = 0
for i in moscow_dic.values():
    sum += len(i)

In [7]:
sum

125